In [1]:
from ipynb.fs.full.metrics import exos_running_time_df
from ipynb.fs.full.metrics import compute_performance
from ipynb.fs.full.metrics import unpickled_results
import pandas as pd
pd.set_option('precision', 2)

import pickle
import os

# Accuracy and Running Time

In [3]:
def get_performance_df(n_streams=2, 
                       bfname = '100K_Case1', 
                       gt_folder = '/home/epanjei/Codes/OutlierGen/exos/nstreams/two',
                       rel_path =  'pickles/nstreams/2',
                       n_experiments=10):
    cwd = os.getcwd()
    gt_filename = f'{n_streams}_{bfname}.pkl'
    result_folder = os.path.join(cwd, rel_path)
    n_experiments = 10
    results = list()
    experiments = list()
    simulation_times = list()
    for i in range(1,n_experiments+1):
        matched, matched2, n_outliers, s_time = compute_performance(gt_folder= gt_folder, 
                                                         gt_filename= gt_filename, 
                                                         result_folder=result_folder, 
                                                         result_filename=f'{i}_{gt_filename}',
                                                         n_streams=n_streams, 
                                                         window_size=1000,
                                                         score_precision = 0.1)
        results.append((len(matched)/n_outliers) * 100)
        simulation_times.append(s_time)
        experiments.append(i)
    accuracy = {'experiment' : experiments, 'accuracy' : results, 'running_time' : simulation_times}
    df = pd.DataFrame(accuracy)
    df.to_pickle(f'{result_folder}/performance_{nstreams}_k1_{bfname}.pkl')
    return df
df = get_performance_df()
df

,experiment,accuracy,running_time
0,1,63.85,27.59
1,2,64.50,28.37
2,3,64.55,27.92
3,4,62.55,28.16
4,5,62.50,29.24
5,6,62.25,28.01
6,7,66.05,28.36
7,8,70.75,28.73
8,9,62.65,29.16
9,10,62.60,29.13
